# Batch Data Processing using Spark on Databricks

Apache Spark is a powerful open-source distributed computing system that provides fast and general-purpose cluster computing for big data processing.

Databricks, on the other hand, is a cloud-based platform built on top of Apache Spark, making it easier to deploy and manage Spark clusters. 
Databricks provides a unified analytics platform that can process large amounts of data quickly. 
Databricks provides an optimised and managed Spark environment.

To clean and query the data from the three Kafka topics, the S3 bucket will be mounted to a Databricks account. Within Databricks three DataFrames will be created to hold this data:

- `df_pin` for the Pinterest post data
- `df_geo` for the geolocation data
- `df_user` for the user data.

This notebook will falcitate the following procedures:

- Load the batch data
- Clean the batch data
- Query the batch data

This notebook uses the ***loading*** methods and dataframe creation methods from the `databricks_load_data` notebook located in the `classes` folder.

This notebook also uses the dataframe cleaning methods from the `databricks_clean_data` file also located in the `classes` folder.

#### Import loading methods 

The following cell allows access to the methods from the `S3DataLoader` class within the `databricks_load_data` notebook.

In [0]:
%run "./classes/databricks_load_data"

#### Import cleaning methods

The following cell allows access to the methods from the `DataCleaning` class within the `databricks_clean_data` notebook

In [0]:
%run "./classes/databricks_clean_data"

#### Instantiate S3DataLoader and DataCleaning

The following cell instantiates the required variables for the `S3DataLoader` class and `DataCleaning` class.

In [0]:
if __name__ == "__main__":
    credentials_path = "dbfs:/user/hive/warehouse/authentication_credentials"
    iam_username = "0ab336d6fcf7"
    topics = ['pin', 'geo', 'user']
    data_loader = S3DataLoader(credentials_path, iam_username, topics)
    data_cleaner = DataCleaning()

## Load Batch Data

Databricks no longer recommends mounting external data locations to Databricks Filesystem.

This notebook has been supplyed as an alternative method to accessing the data.

#### Create Dataframes

The following cell will create three dataframes from the data stored in the S3 bucket.

In [0]:
if __name__ == "__main__":
    data_loader.create_dataframes(mounted=False)

Created DataFrame df_pin from user-0ab336d6fcf7-bucket/topics/0ab336d6fcf7.pin
Created DataFrame df_geo from user-0ab336d6fcf7-bucket/topics/0ab336d6fcf7.geo
Created DataFrame df_user from user-0ab336d6fcf7-bucket/topics/0ab336d6fcf7.user

## Clean Batch Data

#### Clean df.pin

To clean the `df_pin` DataFrame the following cell will perform the following transformations:

- Replace empty entries and entries with no relevant data in each column with `Nones`
- Perform the necessary transformations on the `follower_count` to ensure every entry is a number. Make sure the data type of this column is an `int`.
- Ensure that each column containing numeric data has a `numeric` data type
- Clean the data in the `save_location` column to include only the save location path
- Rename the `index` column to `ind`.
- Reorder the DataFrame columns to have the following column order: (`ind`, `unique_id`, `title`, `description`, `follower_count`, `poster_name`, `tag_list`, `is_image_or_video`, `image_src`, `save_location`, `category`) 

In [0]:
if __name__ == "__main__":
    cleaned_df_pin = data_cleaner.clean_pin_data(df_pin)

    print("Schema for original dataframe")
    df_pin.printSchema()
    print("Schema for cleaned dataframe")
    cleaned_df_pin.printSchema()

    print("Original dataframe:")
    display(df_pin)
    print("Cleaned dataframe:")
    display(cleaned_df_pin)

Schema for original dataframe
root
-- category: string (nullable = true)
-- description: string (nullable = true)
-- downloaded: long (nullable = true)
-- follower_count: string (nullable = true)
-- image_src: string (nullable = true)
-- index: long (nullable = true)
-- is_image_or_video: string (nullable = true)
-- poster_name: string (nullable = true)
-- save_location: string (nullable = true)
-- tag_list: string (nullable = true)
-- title: string (nullable = true)
-- unique_id: string (nullable = true)

Schema for cleaned dataframe
root
-- ind: integer (nullable = true)
-- unique_id: string (nullable = true)
-- title: string (nullable = true)
-- description: string (nullable = true)
-- follower_count: integer (nullable = true)
-- poster_name: string (nullable = true)
-- tag_list: string (nullable = true)
-- is_image_or_video: string (nullable = true)
-- image_src: string (nullable = true)
-- save_location: string (nullable = true)
-- category: string (nullable = true)

Original dataframe:

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
christmas,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",1,5k,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,1706,image,Wear24-7,Local save in /data/christmas,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,b5c8a1b5-9e90-4522-9bec-2477b698d5b7
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
christmas,"Sick of sweeping up pesky needles every single Christmas season? Try keeping up the seasonal cheer this year with this artificial entryway tree, instead. Made with a metal frame…",1,5k,https://i.pinimg.com/originals/45/08/38/4508387c669cb73b0cff865ceba07df4.jpg,2057,image,Wear24-7,Local save in /data/christmas,"Christmas Garden,Farmhouse Christmas Decor,Christmas Home,Outdoor Christmas Trees,Christmas Porch Ideas,Christmas Living Room Decor,Christmas Topiary,Christmas Manger,Christmas Garlands",Green/White Pine Artificial Christmas Tree with 150 Clear/White LightsGreen/White Pine Artificial Christmas Tree with 150 Clear/White Lights - 5' H,883542eb-d365-4033-b857-f5c0530694c4
event-planning,This fabulous DIY project made me drool when I first saw it and I knew immediately that I was going to have to make this! I absolutely love things like this...shiny sparkly thin…,1,985k,https://i.pinimg.com/originals/a6/79/3c/a6793c2e3deebca67ecd82b0087fc13c.jpg,4585,image,"DIY Joy - Crafts, Home Improvement, Decor & Recipes",Local save in /data/event-planning,"Cheap Favors,Wedding Favors Cheap,Wedding Invitations,Wedding Planning On A Budget,Event Planning,Wedding Table Decorations,Wedding Centerpieces,Dollar Tree Centerpieces,Centerpiece Ideas",She Attaches Crystals To A Plate And Creates A Breakfast At Tiffany's Inspired Item!,aa873546-701b-40dd-a339-a3f8aaf78ccb
vehicles,"By David Crane ; defrev (at) gmail (dot) com All photos contained in this article were shot by DefenseReview.com (DR), and are copyrighted. DefenseReview.com owns the copyright…",1,709,https://i.pinimg.com/originals/36/63/12/366312d747da1358397610a86bf21b20.jpg,10538,image,Ricky Lee,Local save in /data/vehicles,"Army Vehicles,Armored Vehicles,Cool Trucks,Cool Cars,Amphibious Vehicle,Offroader,Bug Out Vehicle,Vehicle Wraps,Terrain Vehicle",BC Customs (BCC) Search and Rescue Tactical Vehicle-5 (SRTV-5) Baja Racing-Type All-Terrain Combat Vehicle Armed/Weaponized with 7.62mm NATO Garwood Industries (GI) M134G Minigun/Gatling Gun: SXOR…,5d9fa7e2-2118-4442-99b6-537d60463a6a
event-planning,"Updated: January 25, 2017 You’ve organized some events for your family, friends or community and you have gained a budding reputation for knowing how put events together. You’ve…",1,4k,https://i.pinimg.com/originals/c3/2b/c6/c32bc6ad263857cb0eea19f9cd12beb9.jpg,4357,image,EventPlanning.com | Learn How To Become An Event Planner,Local sa

Cleaned dataframe:

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
1814,61b6cdd3-9f18-4fc7-802b-d47673d63390,46 Stunning Ways to Trim Your Christmas Tree,"Whether coastal, snow-covered, or Charlie Brown-approved, these Christmas tree theme ideas are sure to make your home all the more merry. We'll help you find your true Christmas…",4000000,Better Homes and Gardens,"Christmas Tree Pictures,Creative Christmas Trees,Christmas Tree Inspiration,Gold Christmas Decorations,Beautiful Christmas Trees,Holiday Decor,Christmas Tree Ideas,Candy Cane Christmas Tree,Flocked Christmas Trees Decorated",image,https://i.pinimg.com/originals/51/48/b9/5148b93cf5556fc65459f021dfc41860.jpg,/data/christmas,christmas
6248,8a345a15-c98e-435a-a309-ff5d45f5563c,DIY Farmhouse Media Console Table | The Turquoise Home,This DIY Farmhouse Media Console Table is both beauty and beast. It is gorgeous with the rustic weathered wood and a beast of a storage unit with two sliding barn doors to cover…,205000,The Turquoise Home | Simple DIY + Home Decor Ideas,"Farmhouse Furniture,Farmhouse Decor,Modern Farmhouse,Farmhouse Style,Antique Farmhouse,Farmhouse Design,Console Table Farmhouse,Rustic Media Console,Farmhouse Tv Stand",image,https://i.pinimg.com/originals/7c/90/e5/7c90e550735b582428b777607612d35d.jpg,/data/home-decor,home-decor
1706,b5c8a1b5-9e90-4522-9bec-2477b698d5b7,Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",5000,Wear24-7,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",image,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,/data/christmas,christmas
1740,b43a7094-92ae-42c3-abcd-85df970185bb,"Well, That Was Fun! Let's Talk About December Favourites! - Fashion For Lunch.","Well, That Was Fun! Let’s Talk About December Favourites! Before I start, I just wanted to say I’m a little later than usual publishing this post and I’m really sorry! What can…",21000,Fashion For Lunch,"Front Door Christmas Decorations,Christmas Window Display,Christmas Front Doors,Christmas Swags,Christmas Lights,Outdoor Christmas Garland,Christmas Store Displays,Christmas Windows,Classy Christmas",image,https://i.pinimg.com/originals/bc/a0/43/bca04343a6fdeb7fa6ad18eba850e0fe.jpg,/data/christmas,christmas
3019,ef5e327f-bbed-4c39-849a-3219b8a80654,Our Favorite American Girl Doll DIY Ideas - The Activity Mom,"These are our favorite American Girl Doll DIY Ideas. They are inexpensive, simple to make, and so adorable. Which one will you try first?",42000,The Activity Mom / Blogger of Kids Activities / Teacher / Author,"American Girl Outfits,American Girl Crafts,American Girls,American Girl Stuff,American Girl Storage,American Girl House,American Girl Furniture,Girls Furniture,Baby Doll Furniture",image,https://i.pinimg.com/originals/70/2a/33/702a33d528edc72847f73ba0e3ce837b.jpg,/data/diy-and-crafts,diy-and-crafts
7801,55f282f7-3038-450c-9b72-366072583f1c,"Art Print: Wilson's Be Stronger Than Your Excuses, 32x24in.","Size: 32x24in Be Stronger Than Your ExcusesWe have more Brett Wilson Posters. Choose from our catalog of over 500,000 posters! This art print displays sharp, vivid images with a…",72000,AllPosters,"Motivation Positive,Fitness Motivation Quotes,Motivational Workout Quotes,Quotes About Fitness,Motivational Quotes For Working Out,Health Fitness Quotes,Gym Fitness,Motivating Quotes,Motivational Quotes For Athletes",image,https://i.pinimg.com/originals/d3/49/5e/d3495e71e97c681086ef12fbb10e4eb7.jpg,/data/quotes,quotes
10138,927c4658-cc3f-4b92-9b5c-70743d0c2

#### Clean df.geo

To clean the `df_geo` DataFrame the follwoing cell will perform the following transformations:

- Create a new column `coordinates` that contains an array based on the `latitude` and `longitude` columns
- Drop the `latitude` and `longitude` columns from the DataFrame
- Convert the `timestamp` column from a `string` to a `timestamp` data type
- Reorder the DataFrame columns to have the following column order: (`ind`, `country`, `coordinates`, `timestamp`)

In [0]:
if __name__ == "__main__":
    cleaned_df_geo = data_cleaner.clean_geo_data(df_geo)

    print("Schema for original dataframe")
    df_geo.printSchema()
    print("Schema for cleaned dataframe")
    cleaned_df_geo.printSchema()

    print("Original dataframe:")
    display(df_geo)
    print("Cleaned dataframe:")
    display(cleaned_df_geo)

Schema for original dataframe
root
-- country: string (nullable = true)
-- ind: long (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- timestamp: string (nullable = true)

Schema for cleaned dataframe
root
-- ind: long (nullable = true)
-- country: string (nullable = true)
-- coordinates: array (nullable = false)
 |-- element: double (containsNull = true)
-- timestamp: timestamp (nullable = true)

Original dataframe:

country,ind,latitude,longitude,timestamp
British Indian Ocean Territory (Chagos Archipelago),9314,-42.0773,-163.698,2020-10-20T07:10:24
Antarctica (the territory South of 60 deg S),9185,-10.3764,-22.9809,2019-10-06T18:12:55
Antarctica (the territory South of 60 deg S),9335,-88.4642,-171.061,2020-11-14T23:42:22
Antarctica (the territory South of 60 deg S),7013,-62.92,-63.7974,2019-05-08T11:48:55
Antarctica (the territory South of 60 deg S),7243,-62.92,-63.7974,2020-10-23T19:34:22
United States Minor Outlying Islands,1900,-57.2873,74.8969,2020-06-21T19:45:42
Holy See (Vatican City State),3019,-25.8506,-75.2964,2022-07-07T02:41:54
Holy See (Vatican City State),10316,-12.6955,98.9931,2018-11-24T12:41:53
French Southern Territories,6014,-26.6026,155.206,2019-04-30T12:33:13
French Southern Territories,813,-70.9146,-136.554,2022-06-13T23:08:43


Cleaned dataframe:

ind,country,coordinates,timestamp
8606,Antigua and Barbuda,"List(-88.0974, -172.052)",2021-03-28T14:54:07.000+0000
10119,Christmas Island,"List(-74.5431, -162.795)",2020-10-22T01:59:58.000+0000
10248,Equatorial Guinea,"List(-31.9615, 161.151)",2019-11-03T15:11:00.000+0000
7379,Svalbard & Jan Mayen Islands,"List(48.5847, 19.286)",2019-08-08T04:56:07.000+0000
3763,Bosnia and Herzegovina,"List(-6.78718, 10.54)",2020-01-12T16:30:03.000+0000
3590,British Virgin Islands,"List(-82.4276, -170.019)",2021-03-04T05:19:42.000+0000
5867,Dominican Republic,"List(-63.6774, -118.407)",2019-11-28T12:30:44.000+0000
3019,Holy See (Vatican City State),"List(-25.8506, -75.2964)",2022-07-07T02:41:54.000+0000
6630,Bouvet Island (Bouvetoya),"List(-50.7814, -111.954)",2021-12-12T17:54:13.000+0000
1868,Palestinian Territory,"List(-43.213, 179.303)",2019-09-01T23:09:05.000+0000


#### Clean df.user

To clean the `df_user` DataFrame the following cell will perform the following transformations:

- Create a new column user_name that concatenates the information found in the `first_name` and `last_name` columns
- Drop the `first_name` and `last_name` columns from the DataFrame
- Convert the `date_joined` column from a `string` to a `timestamp` data type
- Reorder the DataFrame columns to have the following column order: (`ind`, `user_name`, `age`, `date_joined`)


In [0]:
if __name__ == "__main__":
    cleaned_df_user = data_cleaner.clean_user_data(df_user)

    print("Schema for original dataframe")
    df_user.printSchema()
    print("Schema for cleaned dataframe")
    cleaned_df_user.printSchema()
    
    print("Original dataframe:")
    display(df_user)
    print("Cleaned dataframe:")
    display(cleaned_df_user)

Schema for original dataframe
root
-- age: long (nullable = true)
-- date_joined: string (nullable = true)
-- first_name: string (nullable = true)
-- ind: long (nullable = true)
-- last_name: string (nullable = true)

Schema for cleaned dataframe
root
-- ind: long (nullable = true)
-- user_name: string (nullable = true)
-- age: long (nullable = true)
-- date_joined: timestamp (nullable = true)

Original dataframe:

age,date_joined,first_name,ind,last_name
59,2017-05-12T21:22:17,Alexander,10673,Cervantes
45,2017-01-08T04:57:20,Jennifer,5773,Harrington
22,2016-03-24T09:38:54,Christopher,1616,Barnett
30,2016-03-02T23:44:12,Christopher,813,Gonzalez
20,2015-12-01T15:08:31,Christopher,5076,Butler
49,2016-04-22T20:36:02,Brittany,10509,Thompson
25,2016-08-19T02:29:26,Elizabeth,10248,English
26,2017-05-14T21:09:12,Jennifer,10842,Gonzalez
20,2015-11-24T21:01:23,Alejandra,6984,Acevedo
43,2016-07-21T15:25:08,Chelsea,10119,Gonzalez


Cleaned dataframe:

ind,user_name,age,date_joined
10673,Alexander Cervantes,59,2017-05-12T21:22:17.000+0000
10552,Michael Hunter,40,2017-05-16T07:09:21.000+0000
10248,Elizabeth English,25,2016-08-19T02:29:26.000+0000
6984,Alejandra Acevedo,20,2015-11-24T21:01:23.000+0000
10344,Brittany Brewer,20,2016-06-26T16:02:23.000+0000
10509,Brittany Thompson,49,2016-04-22T20:36:02.000+0000
813,Christopher Gonzalez,30,2016-03-02T23:44:12.000+0000
9149,Aaron Alexander,21,2015-10-25T07:36:08.000+0000
4315,Michelle Prince,36,2015-12-20T16:38:13.000+0000
1616,Christopher Barnett,22,2016-03-24T09:38:54.000+0000


## Query Batch Data

Before querieing the data the three dataframes (`df_pin`, `df_geo`, and `df_user`) are joined together on the common column heading `ind`.

To make sure that `df_all` is a valid DataFrame it will be created and registered as a temporary table or view before executing any SQL queries. To do this `df_all` is registered as a temporary view using `df_all.createOrReplaceTempView("df_all")`.

However `df_all` is a non-Delta table with many small files. Therefore to improve the performance of queries, `df_all` has been converted to Delta. The new `df_all` table will accelerate queries.

In [0]:
if __name__ == "__main__":
    # Join the three dataframes
    df_all = cleaned_df_pin.join(cleaned_df_geo, 'ind').join(cleaned_df_user, 'ind')

    # Write the Delta table
    delta_table_path = "/delta/my_table/v1"
    df_all.write.format("delta").mode("overwrite").save(delta_table_path)

    # Use DeltaTable API to optimize the Delta table
    delta_table = DeltaTable.forPath(spark, delta_table_path)
    delta_table.vacuum()
    df_all = delta_table.toDF()

    # Create a temperory table of the combined dataframes
    df_all.createOrReplaceTempView("df_all")

    # Display the optimized Delta tables
    display(df_all)

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category,country,coordinates,timestamp,user_name,age,date_joined
1814,61b6cdd3-9f18-4fc7-802b-d47673d63390,46 Stunning Ways to Trim Your Christmas Tree,"Whether coastal, snow-covered, or Charlie Brown-approved, these Christmas tree theme ideas are sure to make your home all the more merry. We'll help you find your true Christmas…",4000000,Better Homes and Gardens,"Christmas Tree Pictures,Creative Christmas Trees,Christmas Tree Inspiration,Gold Christmas Decorations,Beautiful Christmas Trees,Holiday Decor,Christmas Tree Ideas,Candy Cane Christmas Tree,Flocked Christmas Trees Decorated",image,https://i.pinimg.com/originals/51/48/b9/5148b93cf5556fc65459f021dfc41860.jpg,/data/christmas,christmas,Bangladesh,"List(-82.3274, -122.209)",2020-04-25T10:16:28.000+0000,Brian Hansen,31,2016-01-30T13:25:52.000+0000
6248,8a345a15-c98e-435a-a309-ff5d45f5563c,DIY Farmhouse Media Console Table | The Turquoise Home,This DIY Farmhouse Media Console Table is both beauty and beast. It is gorgeous with the rustic weathered wood and a beast of a storage unit with two sliding barn doors to cover…,205000,The Turquoise Home | Simple DIY + Home Decor Ideas,"Farmhouse Furniture,Farmhouse Decor,Modern Farmhouse,Farmhouse Style,Antique Farmhouse,Farmhouse Design,Console Table Farmhouse,Rustic Media Console,Farmhouse Tv Stand",image,https://i.pinimg.com/originals/7c/90/e5/7c90e550735b582428b777607612d35d.jpg,/data/home-decor,home-decor,Bangladesh,"List(-71.4239, -166.014)",2019-11-14T02:37:52.000+0000,David Jackson,29,2016-03-10T17:00:10.000+0000
1706,b5c8a1b5-9e90-4522-9bec-2477b698d5b7,Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",5000,Wear24-7,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",image,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,/data/christmas,christmas,Aruba,"List(-71.5025, -179.257)",2017-11-24T23:36:46.000+0000,Amy Adams,20,2015-10-24T05:05:28.000+0000
1740,b43a7094-92ae-42c3-abcd-85df970185bb,"Well, That Was Fun! Let's Talk About December Favourites! - Fashion For Lunch.","Well, That Was Fun! Let’s Talk About December Favourites! Before I start, I just wanted to say I’m a little later than usual publishing this post and I’m really sorry! What can…",21000,Fashion For Lunch,"Front Door Christmas Decorations,Christmas Window Display,Christmas Front Doors,Christmas Swags,Christmas Lights,Outdoor Christmas Garland,Christmas Store Displays,Christmas Windows,Classy Christmas",image,https://i.pinimg.com/originals/bc/a0/43/bca04343a6fdeb7fa6ad18eba850e0fe.jpg,/data/christmas,christmas,Kuwait,"List(-57.8387, 6.29224)",2019-02-27T22:28:00.000+0000,Richard Stewart,59,2015-11-20T04:51:19.000+0000
3019,ef5e327f-bbed-4c39-849a-3219b8a80654,Our Favorite American Girl Doll DIY Ideas - The Activity Mom,"These are our favorite American Girl Doll DIY Ideas. They are inexpensive, simple to make, and so adorable. Which one will you try first?",42000,The Activity Mom / Blogger of Kids Activities / Teacher / Author,"American Girl Outfits,American Girl Crafts,American Girls,American Girl Stuff,American Girl Storage,American Girl House,American Girl Furniture,Girls Furniture,Baby Doll Furniture",image,https://i.pinimg.com/originals/70/2a/33/702a33d528edc72847f73ba0e3ce837b.jpg,/data/diy-and-crafts,diy-and-crafts,Holy See (Vatican City State),"List(-25.8506, -75.2964)",2022-07-07T02:41:54.000+0000,Hector Clark,29,2017-06-27T00:45:56.000+0000
7801,55f282f7-3038-450c-9b72-3660725

#### Question 1: Find the most popular category in each country

- Find the most popular Pinterest category people post to based on their country.
- The query should return a DataFrame that contains the following columns: (`country`, `category`, `category_count`)


In [0]:
top_category_per_country = spark.sql("""
    WITH ranked_categories AS (
        SELECT
            country, 
            category, 
            COUNT(category) AS category_count, 
            RANK() OVER (PARTITION BY country ORDER BY count(category) DESC) AS category_rank 
        FROM df_all 
        GROUP BY country, category 
    ) 
    SELECT country, category, category_count 
    FROM ranked_categories 
    WHERE category_rank = 1 
    ORDER BY country
""")
display(top_category_per_country)

country,category,category_count
Afghanistan,finance,2
Albania,art,6
Algeria,quotes,5
American Samoa,travel,3
American Samoa,home-decor,3
Andorra,tattoos,5
Angola,diy-and-crafts,2
Anguilla,beauty,2
Anguilla,diy-and-crafts,2
Antarctica (the territory South of 60 deg S),tattoos,2


#### Question 2: Find which was the most popular category each year

- Find how many posts each category had between 2018 and 2022.
- The query will return a DataFrame that contains the following columns: (`post_year`, `category`, `category_count`)


In [0]:
top_category_per_year = spark.sql("""
    WITH ranked_categories AS (
        SELECT
            category,
            EXTRACT(YEAR FROM timestamp) AS post_year,
            COUNT(category) AS category_count,
            RANK() OVER (PARTITION BY EXTRACT(YEAR FROM timestamp) ORDER BY count(category) DESC) AS category_rank 
        FROM df_all 
        GROUP BY EXTRACT(YEAR FROM timestamp), category
    )
    SELECT post_year, category, category_count 
    FROM ranked_categories 
    WHERE category_rank = 1 AND post_year BETWEEN 2018 AND 2022
    ORDER BY post_year
""")
display(top_category_per_year)

post_year,category,category_count
2018,tattoos,9
2019,finance,9
2020,event-planning,8
2020,home-decor,8
2021,finance,10
2022,beauty,5
2022,mens-fashion,5


#### Question 3: Find the user with most followers in each country

- Step 1: For each country find the user with the most followers.
  - Your query should return a DataFrame that contains the following columns: (`country`, `poster_name`, `follower_count`)
- Step 2: Based on the above query, find the country with the user with most followers.
  - Your query should return a DataFrame that contains the following columns: (`country`, `follower_count`)
  - This DataFrame should have only one entry.

In [0]:
# Step 1:
top_user_per_country = spark.sql("""
    SELECT country, poster_name, MAX(follower_count) AS follower_count
    FROM df_all
    GROUP BY country, poster_name
    ORDER BY follower_count DESC
""")
display(top_user_per_country)

# Step 2:
country_with_top_user = spark.sql("""
    WITH top_users AS (
        SELECT country, poster_name, MAX(follower_count) AS follower_count
        FROM df_all
        GROUP BY country, poster_name
        ORDER BY follower_count DESC
    )
    SELECT country, follower_count
    FROM top_users
    ORDER BY follower_count DESC
    LIMIT 1
""")
display(country_with_top_user)


country,poster_name,follower_count
Angola,Tastemade,8000000
American Samoa,Mamas Uncut,8000000
American Samoa,BuzzFeed,5000000
Algeria,Apartment Therapy,5000000
Albania,The Minds Journal,5000000
Bangladesh,Better Homes and Gardens,4000000
Chile,Target,4000000
Afghanistan,9GAG,3000000
Botswana,OkChicas,3000000
Christmas Island,Instructables,3000000


country,follower_count
Angola,8000000


#### Question 4: Find the most popular category for different age groups

- What is the most popular category people post to based on the following age groups: (`18-24`, `25-35`, `36-50`, `+50`)
- The query should return a DataFrame that contains the following columns: (`age_group`, `category`, `category_count`)

In [0]:
top_category_per_age = spark.sql("""
    WITH age_groups AS (
        SELECT 
            CASE 
                WHEN Age BETWEEN 18 AND 24 THEN '18-24'
                WHEN Age BETWEEN 25 AND 35 THEN '25-35'
                WHEN Age BETWEEN 36 AND 50 THEN '36-50'
                WHEN Age >= 51 THEN '50+'
                ELSE 'NA'
            END AS age_group,
            category, 
            COUNT(category) AS category_count
        FROM df_all
        GROUP BY age_group, category
    ),
    ranked_ages AS (
        SELECT 
            age_group, 
            category, 
            category_count, 
            RANK() OVER (PARTITION BY age_group ORDER BY category_count DESC) AS category_rank
        FROM age_groups
    )
    SELECT age_group, category, category_count
    FROM ranked_ages
    WHERE category_rank = 1
""")
display(top_category_per_age)


age_group,category,category_count
18-24,tattoos,19
25-35,home-decor,13
36-50,vehicles,8
50+,event-planning,6


#### Question 5: Find the median follower count for different age groups

- What is the median follower count for users in the following age groups: (`18-24`, `25-35`, `36-50`, `+50`)
- The query should return a DataFrame that contains the following columns: (`age_group`, `median_follower_count`)


In [0]:
median_follower_per_age_group = spark.sql("""
    SELECT 
        CASE 
            WHEN Age BETWEEN 18 AND 24 THEN '18-24'
            WHEN Age BETWEEN 25 AND 35 THEN '25-35'
            WHEN Age BETWEEN 36 AND 50 THEN '36-50'
            WHEN Age >= 51 then '50+'
            ELSE 'NA'
        End as age_group,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count 
    FROM df_all
    GROUP BY age_group
    ORDER BY median_follower_count DESC
""")
display(median_follower_per_age_group)                                                                

age_group,median_follower_count
18-24,59500.0
25-35,37000.0
50+,9000.0
36-50,5500.0


#### Question 6: Find how many users have joined each year?

- Find how many users have joined between 2015 and 2020.
- The query should return a DataFrame that contains the following columns: (`post_year`, `number_users_joined`)

In [0]:
users_joined_per_year = spark.sql("""
    SELECT
        EXTRACT(YEAR FROM date_joined) AS post_year,
        COUNT(DISTINCT(user_name)) AS number_users_joined
    FROM df_all 
    WHERE EXTRACT(YEAR FROM date_joined) BETWEEN 2015 AND 2020
    GROUP BY post_year
""")
display(users_joined_per_year)

post_year,number_users_joined
2015,97
2016,120
2017,56


#### Question 7: Find the median follower count of users based on their joining year

- Find the median follower count of users have joined between 2015 and 2020.
- Your query should return a DataFrame that contains the following columns: (`post_year`, `median_follower_count`)

In [0]:
median_follower_per_join_year = spark.sql("""
    SELECT
        EXTRACT(YEAR FROM date_joined) AS post_year,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
    FROM df_all 
    WHERE EXTRACT(YEAR FROM date_joined) BETWEEN 2015 AND 2020
    GROUP BY post_year
""")
display(median_follower_per_join_year)

post_year,median_follower_count
2015,69000.0
2016,25000.0
2017,5500.0


#### Question 8: Find the median follower count of users based on their joining year and age group

- Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.
- The query should return a DataFrame that contains the following columns: (`age_group`, `post_year`, `median_follower_count`)

In [0]:
median_follower_per_join_year_age_group = spark.sql("""
    SELECT
        CASE 
            WHEN Age BETWEEN 18 AND 24 THEN '18-24'
            WHEN Age BETWEEN 25 AND 35 THEN '25-35'
            WHEN Age BETWEEN 36 AND 50 THEN '36-50'
            WHEN Age >= 51 then '50+'
            ELSE 'NA'
        End as age_group,
        EXTRACT(YEAR FROM date_joined) AS post_year,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
    FROM df_all 
    WHERE EXTRACT(YEAR FROM date_joined) BETWEEN 2015 AND 2020
    GROUP BY age_group, post_year
    Order by age_group, post_year
""")
display(median_follower_per_join_year_age_group)

age_group,post_year,median_follower_count
18-24,2015,243000.0
18-24,2016,43500.0
18-24,2017,7500.0
25-35,2015,53500.0
25-35,2016,31000.0
25-35,2017,5000.0
36-50,2015,6000.0
36-50,2016,5000.0
36-50,2017,6000.0
50+,2015,25500.0
